In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 0 ▶ Install required libraries
!pip install \
    imagecodecs \
    tifffile \
    rasterio \
    timm \
    segmentation-models-pytorch \
    matplotlib \
    tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Cell 1 ▶ Imports & Drive mount
import imagecodecs          # ensure TIFF LZW support
import os, math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import segmentation_models_pytorch as smp
from tifffile import imread
from tqdm.auto import tqdm
from google.colab import drive


In [ ]:
# Cell 2 ▶ Dataset for TransUNet testing (image + 224² LST target)
class LSTPatchDataset(Dataset):
    def __init__(self, df, patches_dir, transform):
        self.df          = df.reset_index(drop=True)
        self.patches_dir = patches_dir
        self.tfm         = transform

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        arr = imread(os.path.join(self.patches_dir, row.patch_filename)
                    ).astype(np.float32)            # (4,H,W)

        img  = self.tfm(arr[[1,2,3]].transpose(1,2,0).astype(np.uint8))
        lst  = torch.tensor(arr[0], dtype=torch.float32).unsqueeze(0)
        lst  = F.interpolate(lst.unsqueeze(0), size=(224,224),
                             mode='bilinear', align_corners=False
                            ).squeeze(0)            # [1,224,224]
        lst  = torch.clamp(lst, 0.0, 1.0)           # match training scale
        return img, lst


In [ ]:
# Cell 3 ▶ Build 2023 test DataLoader
CSV       = "/content/drive/MyDrive/test_patches/patch_with_meteo_2023_last6h.csv"
PATCH_DIR = "/content/drive/MyDrive/test_patches"

df_test = pd.read_csv(CSV)          # we only need patch_filename column
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_ds = LSTPatchDataset(df_test, PATCH_DIR, transform)
test_dl = DataLoader(test_ds,
                     batch_size=4,
                     shuffle=False,
                     num_workers=0,   # main proc ⇒ uses LZW codec
                     pin_memory=True)
print(f"Test patches: {len(test_ds)}")


Test patches: 3430


In [ ]:
# Cell 4 ▶ Re-create TransUNet model & load best weights
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = smp.Unet(
    encoder_name='mit_b0',     # same backbone as training
    encoder_weights=None,      # weights come from your .pth file
    in_channels=3,
    classes=1
).to(DEVICE)

ckpt_path = "/content/drive/MyDrive/best_transunet_ftl.pth"
model.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
model.eval()
print("✅ loaded checkpoint:", ckpt_path)


✅ loaded checkpoint: /content/drive/MyDrive/best_transunet_ftl.pth


In [ ]:
# Cell 5 ▶ Loss & quick evaluation on first 20 batches
def focal_tversky_loss(logits, targets,
                       alpha=0.7, beta=0.3, gamma=0.75, eps=1e-6):
    probs = torch.sigmoid(logits)
    dims  = (2,3)
    TP = (probs * targets).sum(dim=dims)
    FN = ((1-probs) * targets).sum(dim=dims)
    FP = (probs * (1-targets)).sum(dim=dims)
    tversky = (TP + eps) / (TP + alpha*FN + beta*FP + eps)
    return ((1 - tversky) ** gamma).mean()

max_batches = 50
ftl_total   = 0.0
rmse_total  = 0.0
pix_total   = 0

for imgs, masks in tqdm(
        iter(test_dl), total=max_batches, desc="Testing (20 batches)"
    ):
    if max_batches == 0: break
    imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
    with torch.no_grad():
        preds = model(imgs)
    # FTL
    ftl = focal_tversky_loss(preds, masks).item()
    ftl_total += ftl

    # RMSE (after sigmoid because training logits weren’t clamped)
    preds_float = torch.sigmoid(preds)
    rmse_batch = ((preds_float - masks) ** 2).sum().item()
    rmse_total += rmse_batch
    pix_total  += masks.numel()

    max_batches -= 1
    if max_batches == 0:
        break

rmse = math.sqrt(rmse_total / pix_total)
print(f"\n▶ Avg Focal-Tversky on 50 batches: {ftl_total/20:.4f}")
print(f"▶ RMSE on 50 batches: {rmse:.3f} (scaled LST units)")


Testing (20 batches):   0%|          | 0/50 [00:00<?, ?it/s]


▶ Avg Focal-Tversky on 50 batches: 0.5521
▶ RMSE on 50 batches: 0.171 (scaled LST units)
